In [37]:
import tensorflow as tf 
from tensorflow.keras.models import load_model  
import itertools

model = load_model('./mobilenet_combination_model_fold_4bp_2.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer1 (InputLayer)   [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_layer2 (InputLayer)   [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)           2257984   ['input_layer1[0][0]',        
 tional)                                                             'input_layer2[0][0]']        
                                                                                                  
 conv2d (Conv2D)             (None, 7, 7, 32)             40992     ['mobilenetv2_1.00_224[0][

In [38]:
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

def preprocess_image(img_path):
        img = load_img(img_path, color_mode='grayscale', target_size=(224, 224))
        img_array = img_to_array(img)
        pseudo_rgb_img = np.repeat(img_array, 3, axis=2)
        return preprocess_input(pseudo_rgb_img)

In [39]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageDraw, ImageFont

# Function to compute Grad-CAM
def make_gradcam_heatmap(input_data, model, last_conv_layer_name, pred_class_idx):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Forward pass through the model to get activations and predictions
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_data)
        if pred_class_idx == 1:
            class_channel = predictions[:, 0]
        else:
            class_channel = 1 - predictions[:, 0]

    # Compute the gradient of the output with respect to the conv layer output
    grads = tape.gradient(class_channel, conv_outputs)

    # Pool the gradients over the spatial dimensions
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the output of the conv layer with the pooled gradients
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

from tensorflow.keras.preprocessing.image import array_to_img

def superimpose_gradcam_on_image(heatmap, img_array, alpha=0.4):
    import matplotlib.cm as cm

    heatmap = tf.image.resize(heatmap[..., tf.newaxis], (img_array.shape[0], img_array.shape[1]))
    heatmap = tf.squeeze(heatmap)
    heatmap = heatmap.numpy()

    # Normalize heatmap between 0 and 1
    heatmap = np.uint8(255 * heatmap)

    # Apply the colormap (e.g., 'jet') to the heatmap
    colormap = cm.get_cmap('jet')
    colored_heatmap = colormap(heatmap)
    colored_heatmap = np.uint8(255 * colored_heatmap[:, :, :3])  # Discard alpha channel

    img_array = np.uint8(255 * (img_array - img_array.min()) / (img_array.max() - img_array.min()))

    # Superimpose the heatmap on the original image
    superimposed_img = colored_heatmap * alpha + img_array * (1 - alpha)
    superimposed_img = np.uint8(superimposed_img)
    superimposed_img = array_to_img(superimposed_img)

    return superimposed_img

img_root_path = 'directory_of_images'

for root, dirs, files in os.walk(img_root_path):
    print(f"Directories in {root}: {dirs}")
    print(f"Files in {root}: {files}")

for root, dirs, files in os.walk(img_root_path):
    print(root)
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(root, file)
            print(img_path)
            
            input_2 = preprocess_image(img_path)
            input_2 = np.expand_dims(input_2, axis=0)
            
            pred = model.predict([input_2, input_2])
            print(f"Model prediction output for {file}:", pred)
            
            pred_value = pred[0][0]
            
            if pred_value < 0.5:
                predicted_label = 'normal'
                pred_class_idx = 0
            else:
                predicted_label = 'abnormal'
                pred_class_idx = 1
            
            last_conv_layer_name = 'dropout'  
            
            # Generate Grad-CAM heatmap
            heatmap = make_gradcam_heatmap(
                [input_2, input_2], model, last_conv_layer_name, pred_class_idx=pred_class_idx
            )
            
            # Superimpose the heatmap on the original input image
            gradcam_img = superimpose_gradcam_on_image(heatmap, input_2[0])
            
            draw = ImageDraw.Draw(gradcam_img)
            
            try:
                font = ImageFont.truetype("arial.ttf", 20)
            except IOError:
                font = ImageFont.load_default()
            
            text_position = (10, 10)
            
            draw.text(text_position, predicted_label, fill='white', font=font)
            
            subdirectory_name = os.path.basename(root)
            save_filename = f'{subdirectory_name}.png'
            save_path = os.path.join(root, save_filename)
            gradcam_img.save(save_path)
            
            print(f"Grad-CAM image saved at: {save_path}")

    


Directories in ../Downloads/: ['thorax_clean']
Files in ../Downloads/: ['Steven-Halim_-Felix-Halim-Competitive-Programming-3_-The-New-Lower-Bound-of-Programming-Contests-Lulu.com-_2013_.pdf', 'thorax_clean-20241011T101031Z-001.zip', '(2) Steven Halim, Felix Halim, Suhendry Effendy - Competitive Programming 4 - Book 2-lulu (2022).pdf']
Directories in ../Downloads/thorax_clean: ['Thorax-231023-2310230137_Series_1001_0000', 'Thorax-231023-2310230012_Series_1001_0000', 'Thorax-231030-2310300152_Series_1001_0000', 'Thorax-231028-2310280100_Series_1001_0000', 'Thorax-231030-2310300170_Series_1001_0000', 'Thorax-231023-2310230121_Series_1001_0000', 'Thorax-231027-2310270151_Series_1001_0000', 'Thorax-231027-2310270010_Series_1001_0000', 'Thorax-231021_231022_Pneumonia-2310210103_Series_1001_0000', 'Thorax-231023-2310230099_Series_1001_0000', 'Thorax-231030-2310300135_Series_1001_0000', 'Thorax-231021_231022_Pneumonia-2310220023_Series_1001_0000', 'Thorax-231023-2310230197_Series_1001_0000', '

/tmp/ipykernel_81245/1763766341.py:52: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('jet')


Model prediction output for Thorax-231023-2310230137_Series_1001_0000-43.jpg: [[0.9916619]]
Predicted label for Thorax-231023-2310230137_Series_1001_0000-43.jpg: abnormal
Grad-CAM image saved at: ../Downloads/thorax_clean/Thorax-231023-2310230137_Series_1001_0000/Thorax-231023-2310230137_Series_1001_0000.png
../Downloads/thorax_clean/Thorax-231023-2310230012_Series_1001_0000
../Downloads/thorax_clean/Thorax-231023-2310230012_Series_1001_0000/Thorax-231023-2310230012_Series_1001_0000.png
1/1 [==============================] - 0s 92ms/step
Model prediction output for Thorax-231023-2310230012_Series_1001_0000.png: [[0.9990014]]
Predicted label for Thorax-231023-2310230012_Series_1001_0000.png: abnormal
Grad-CAM image saved at: ../Downloads/thorax_clean/Thorax-231023-2310230012_Series_1001_0000/Thorax-231023-2310230012_Series_1001_0000.png
../Downloads/thorax_clean/Thorax-231023-2310230012_Series_1001_0000/Thorax-231023-2310230012_Series_1001_0000-4.jpg
1/1 [==============================]

/tmp/ipykernel_81245/1763766341.py:49: RuntimeWarning: invalid value encountered in cast
  heatmap = np.uint8(255 * heatmap)


Model prediction output for Thorax-231023-2310230064_Series_1001_0000.png: [[0.9901096]]
Predicted label for Thorax-231023-2310230064_Series_1001_0000.png: abnormal
Grad-CAM image saved at: ../Downloads/thorax_clean/Thorax-231023-2310230064_Series_1001_0000/Thorax-231023-2310230064_Series_1001_0000.png
../Downloads/thorax_clean/Thorax-231023-2310230064_Series_1001_0000/Thorax-231023-2310230064_Series_1001_0000-17.jpg
1/1 [==============================] - 0s 91ms/step
Model prediction output for Thorax-231023-2310230064_Series_1001_0000-17.jpg: [[0.05170694]]
Predicted label for Thorax-231023-2310230064_Series_1001_0000-17.jpg: normal
Grad-CAM image saved at: ../Downloads/thorax_clean/Thorax-231023-2310230064_Series_1001_0000/Thorax-231023-2310230064_Series_1001_0000.png
../Downloads/thorax_clean/Thorax-231024-2310240168_Series_1001_0000
../Downloads/thorax_clean/Thorax-231024-2310240168_Series_1001_0000/Thorax-231024-2310240168_Series_1001_0000.png
1/1 [==============================]